In [1]:
import sys
sys.path.insert(0,'..')

from light_pipe import Data, Parallelizer, Transformer, mercantile

In [2]:
import json


def get_filepaths(file_manifest_path, path_extension, *args, **kwargs):
    with open(file_manifest_path, "r") as fp:
        files_dict = json.load(fp)

    for geojson_path, path_tuples in files_dict.items():
        for (img_path, udm_path) in path_tuples:
            img_path = path_extension + img_path[1:]
            udm_path = path_extension + udm_path[1:]
            yield (geojson_path, img_path, udm_path)

In [3]:
def get_tiles(paths_tup, zooms, truncate=True, *args, **kwargs):
    geojson_path, img_path, udm_path = paths_tup
    with open(geojson_path, "r") as fp:
        geojson_dict = json.load(fp)    
    geo_bounds = mercantile.geojson_bounds(geojson_dict)
    west = geo_bounds.west
    south = geo_bounds.south
    east = geo_bounds.east
    north = geo_bounds.north

    tiles = mercantile.tiles(west, south, east, north, zooms, truncate)
    for tile in tiles:
        yield tile, geojson_path, img_path, udm_path

In [4]:
file_manifest_path = "C:/Projects/therm-two/data/files.json"
path_extension = "C:\\Projects\\therm-two"

zooms = [16, 17, 18, 19, 20, 21, 22]
truncate = True

data = Data(generator=get_filepaths)
pipe = data | Transformer(get_tiles, zooms=zooms)

In [19]:
import os
from light_pipe import gridding


def make_tile_datasets(item, out_dir, pixel_x_meters, pixel_y_meters, is_archive, *args, **kwargs):
    tile, geojson_path, img_path, udm_path = item

    if is_archive:
        img_path = '/vsizip/' + img_path
        udm_path = '/vsizip/' + udm_path

    zoom = tile.z

    quad_key = mercantile.quadkey(tile)

    # Make filepaths
    out_sub_dir = os.path.join(out_dir, "zoom_" + str(zoom), quad_key + '/')
    os.makedirs(out_sub_dir, exist_ok=True)
    out_target_path = os.path.join(out_sub_dir, "target.tif")
    out_img_path = os.path.join(out_sub_dir, "img.tif")
    out_udm_path = os.path.join(out_sub_dir, "udm.tif")


    # Make datasets
    gridding.make_grid_cell_dataset(
        grid_cell=tile, datum=geojson_path, return_filepaths=True, is_label=True, 
        in_memory=False, pixel_x_meters=pixel_x_meters, pixel_y_meters=pixel_y_meters,
        grid_cell_filepath=out_target_path
    )

    gridding.make_grid_cell_dataset(
        grid_cell=tile, datum=img_path, return_filepaths=True, is_label=False, 
        in_memory=False, pixel_x_meters=pixel_x_meters, pixel_y_meters=pixel_y_meters,
        grid_cell_filepath=out_img_path
    )

    gridding.make_grid_cell_dataset(
        grid_cell=tile, datum=udm_path, return_filepaths=True, is_label=True, 
        in_memory=False, pixel_x_meters=pixel_x_meters, pixel_y_meters=pixel_y_meters,
        grid_cell_filepath=out_udm_path
    )

    return out_target_path, out_img_path, out_udm_path

In [20]:
file_manifest_path = "C:/Projects/therm-two/data/files.json"
path_extension = "C:\\Projects\\therm-two"
out_dir = '../data/samples/'
pixel_x_meters = 3.0
pixel_y_meters = -3.0
is_archive = True

zooms = [16, 17, 18, 19, 20, 21, 22]
truncate = True

data = Data(generator=get_filepaths)
pipe = data | Transformer(get_tiles, zooms=zooms)
pipe = pipe | Transformer(make_tile_datasets, out_dir=out_dir, pixel_x_meters=pixel_x_meters, pixel_y_meters=pixel_y_meters, is_archive=is_archive)

In [21]:
items = pipe(file_manifest_path=file_manifest_path, path_extension=path_extension, block=True)

In [22]:
items

[('../data/samples/zoom_16\\2101313120102101/target.tif',
  '../data/samples/zoom_16\\2101313120102101/img.tif',
  '../data/samples/zoom_16\\2101313120102101/udm.tif'),
 ('../data/samples/zoom_17\\21013131201021011/target.tif',
  '../data/samples/zoom_17\\21013131201021011/img.tif',
  '../data/samples/zoom_17\\21013131201021011/udm.tif'),
 ('../data/samples/zoom_17\\21013131201021013/target.tif',
  '../data/samples/zoom_17\\21013131201021013/img.tif',
  '../data/samples/zoom_17\\21013131201021013/udm.tif'),
 ('../data/samples/zoom_18\\210131312010210112/target.tif',
  '../data/samples/zoom_18\\210131312010210112/img.tif',
  '../data/samples/zoom_18\\210131312010210112/udm.tif'),
 ('../data/samples/zoom_18\\210131312010210130/target.tif',
  '../data/samples/zoom_18\\210131312010210130/img.tif',
  '../data/samples/zoom_18\\210131312010210130/udm.tif'),
 ('../data/samples/zoom_19\\2101313120102101122/target.tif',
  '../data/samples/zoom_19\\2101313120102101122/img.tif',
  '../data/samples